# Fetch Reward DA Exercise
Question #3 : Detecting data issue (null/missing value, duplicate and data inconsistency)
<br>
Author: Chengwu Weng
<br>
Date: 8/26/2021

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [3]:
data_users = pd.read_csv('users.csv')
data_receipts = pd.read_csv('receipts.csv')
data_brands = pd.read_csv('brands.csv')
data_rewardsReceiptItemList = pd.read_csv('rewardsReceiptItemList.csv')

# Data Issue


*** 
I found mainly three data issues: null value, duplicate value & data inconsistency.
<br>

***


## Null Value

In [4]:
data_users.isna().sum()

_id              0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64

In [5]:
data_receipts.isna().sum()

_id                          0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                   0
pointsAwardedDate          582
pointsEarned               510
purchaseDate               448
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
dtype: int64

In [6]:
data_brands.isna().sum()

_id               0
barcode           0
category        155
categoryCode    650
cpg               0
name              0
topBrand        612
cpg_id            0
cpg_ref           0
brandCode       269
dtype: int64

In [7]:
data_rewardsReceiptItemList.isna().sum()

preventTargetGapPoints                6583
pointsPayerId                         5674
userFlaggedDescription                6787
itemPrice                              174
originalMetaBriteQuantityPurchased    6926
pointsNotAwardedReason                6601
barcode                               3851
needsFetchReviewReason                6722
originalMetaBriteDescription          6931
userFlaggedBarcode                    6604
metabriteCampaignId                   6078
partnerItemId                            0
userFlaggedQuantity                   6642
originalFinalPrice                    6932
pointsEarned                          6014
quantityPurchased                      174
originalMetaBriteBarcode              6917
priceAfterCoupon                      5985
userFlaggedNewItem                    6618
competitorRewardsGroup                6666
receiptid                                0
rewardsGroup                          5210
discountedItemPrice                   1172
originalMet

## Duplicate Value

### Users


*** 
**One row should represent one users.**
<br>
However, in the table of `Users`, there are `209` duplicates out of total rows of 413 except 82 fetch test staff. 

***


In [8]:
print('total value of duplicate in all user id', data_users['_id'].duplicated().sum())
user_con = data_users[data_users.role != 'fetch-staff' ]
print('total value of duplicate in consumer user id', user_con['_id'].duplicated().sum())

total value of duplicate in all user id 283
total value of duplicate in consumer user id 209


In [9]:
user_con[user_con._id.duplicated()].head(5)

,_id,active,createdDate,lastLogin,role,signUpSource,state
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 07:24:04.800000,2021-01-03 07:25:37.858000,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 07:24:04.800000,2021-01-03 07:25:37.858000,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 07:24:04.800000,2021-01-03 07:25:37.858000,consumer,Email,WI
5,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 07:24:04.800000,2021-01-03 07:25:37.858000,consumer,Email,WI
8,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 07:24:04.800000,2021-01-03 07:25:37.858000,consumer,Email,WI


### Brands


*** 
**Same barcode is for same product.**
<br>
However, in the table of `Brands`, one barcode has two different category as well as brand names.

Our program website can be found at <a href="http://earthdatascience.org" target="_blank">this link</a>.

***


In [10]:
data_brands._id.duplicated().sum()

0

In [11]:
data_brands[data_brands.barcode.duplicated()].head(2)

,_id,barcode,category,categoryCode,cpg,name,topBrand,cpg_id,cpg_ref,brandCode
299,5a8c33f3e4b07f0a2dac8943,511111504139,Grocery,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5a734034e4b0...",Pace,False,5a734034e4b0d58f376be874,Cogs,PACE
412,5ccb2ece166eb31bbbadccbe,511111504788,Condiments & Sauces,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '559c2234e4b0...",The Pioneer Woman,NaN,559c2234e4b06aca36af13c6,Cogs,PIONEER WOMAN


In [12]:
data_brands[data_brands.barcode == '511111504139']

,_id,barcode,category,categoryCode,cpg,name,topBrand,cpg_id,cpg_ref,brandCode


### Receipts & Reward Receipt Item List


*** 
**One receipt should have one or many different items.**
<br><br>
However, there are many same items in one receipt with the same attributes in rewardsReceiptItemList like barcode, description & quantityPurchased etc.
<br><br>
The meaning behind it is too vague to understand. We cannot tell it if a user purchased many same items or just duplicates issue. But if someone purchased many same products, we should use the attribute of quantityPurchased to represent it instead of using duplicates.


***


In [13]:
data_receipts['_id'].duplicated().sum()

0

In [14]:
data_rewardsReceiptItemList['receiptid'].duplicated().sum()

6262

In [15]:
data_receipts.rewardsReceiptItemList[12] # with receipt id = '5ff1e1b60a7214ada100055c' and barcode = '034100573065'

"[{'barcode': '034100573065', 'description': 'MILLER LITE 24 PACK 12OZ CAN', 'finalPrice': '29', 'itemPrice': '29', 'partnerItemId': '1', 'pointsEarned': '870.0', 'pointsPayerId': '5332f709e4b03c9a25efd0f1', 'quantityPurchased': 1, 'rewardsGroup': 'MILLER LITE 24 PACK', 'rewardsProductPartnerId': '5332f709e4b03c9a25efd0f1', 'targetPrice': '77', 'receiptid': '5ff1e1b60a7214ada100055c'}, {'barcode': '034100573065', 'description': 'MILLER LITE 24 PACK 12OZ CAN', 'finalPrice': '29', 'itemPrice': '29', 'partnerItemId': '2', 'pointsEarned': '870.0', 'pointsPayerId': '5332f709e4b03c9a25efd0f1', 'quantityPurchased': 1, 'rewardsGroup': 'MILLER LITE 24 PACK', 'rewardsProductPartnerId': '5332f709e4b03c9a25efd0f1', 'targetPrice': '77', 'receiptid': '5ff1e1b60a7214ada100055c'}, {'barcode': '034100573065', 'description': 'MILLER LITE 24 PACK 12OZ CAN', 'finalPrice': '29', 'itemPrice': '29', 'partnerItemId': '3', 'pointsEarned': '870.0', 'pointsPayerId': '5332f709e4b03c9a25efd0f1', 'quantityPurchased

## Inconsistency between user in receipts and users


*** 
**Users in receipts should be included in users table**
<br>
However, there is user inconsistency, meaning that users in receipts table are more than users in users table.
<br>

***


In [16]:
receipts_customers = data_receipts['userId'].unique().tolist()

In [17]:
users_customers = user_con['_id'].unique().tolist() # only include consumer users

In [18]:
len(receipts_customers) - len(users_customers)

54

In [19]:
# print users in receipt but not in user
for customer in receipts_customers:
    if customer not in users_customers:
        print(customer)

5f9c74f7c88c1415cbddb839
5ff1e1e9b6a9d73a3a9f10f6
5ff1e1dfcfcf6c399c274ab3
5f9c74e3f1937815bd2c1d73
5ff1e196cfcf6c399c274a38
54943462e4b07e684157a532
5ff36d78135e7011bcb86488
5964eb07e4b03efd0c0f267b
5c3388caea88e15513a95069
5ff37124135e7011bcb86bc3
5fbc35711d967d1222cbfefc
5ff473f3c1e2d0121a9b2707
5ff4ce68c1e2d0121a9b3022
5fa5ad376a26f611e71ab5ef
5ff4ce65c3d63511e2a4853b
5ff4ce3cc1e2d0121a9b2fba
5ff5d19b8f142f11dd188696
5ff73be5eb7c7d31ca8a45a7
5ff7741e04929111f6e90902
5ff79459b3348b11c933736d
5ff75004b3348b11c9336bd6
5ff73be9eb7c7d31ca8a45bc
5ff7946004929111f6e90ceb
5ff726a38f142f11dd1895dc
5ff8da4e04929111f6e91462
5ff8da7c04929111f6e914de
5ff8da65b3348b11c9337b29
5ff8da62b3348b11c9337b21
5fa5b0b720dc5111dd86dcc1
5ff8c88604929111f6e913b7
5ff873d304929111f6e91096
5ffc98b604929111f6e923f5
5ffc9d8cb3348b11c9338927
5ffc9da0b3348b11c9338951
5ffc8cc104929111f6e922a3
5ffcb4b9b3348b11c9338ae7
5ffca2f3b3348b11c9338a10
5ffc92a104929111f6e923cb
5ffcb48d04929111f6e92594
5ffc9d9d04929111f6e9244f
